In [4]:
from openai import OpenAI
import os
import json
from dotenv import load_dotenv
from pathlib import Path
from pydantic import BaseModel

extracted_answers = []
query_file = "../queries.json"


dotenv_path = Path('../../global_environment.env')
load_dotenv(dotenv_path=dotenv_path)
openai_key = os.getenv('OPENAI_KEY')

client = OpenAI(
        api_key=openai_key
)

if(openai_key):
    print('Key loaded successfully')
else:
    print('API Key not found')

Key loaded successfully


In [5]:
class Category(BaseModel):
    category: str
    isMatch: bool
    percentageMatch: float
    keywords: list[str]

class SignalTags(BaseModel):
    signal_index: str
    categories: list[Category]

def getCategories(signal):
    signal_index = signal['signal_index']
    print('Extracting from ', signal_index)
    with open(query_file, 'r') as f:
        queries = json.load(f)
    descriptions = signal['associative_descriptions']
    desc_list = '\n'.join(descriptions)
    response = client.beta.chat.completions.parse(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are an assistant that processes descriptions of haptic feedback and returns json objects"},
                {"role": "user", "content":"Signal index: "+str(signal_index)+":\n"+ queries['tagPrompt']+'\n'+desc_list},
                ],
                response_format=SignalTags
        )
    print(response.choices[0].message.content)
    return json.loads(response.choices[0].message.content)

with open('../../Data/signal-descriptions/collated_descriptions.json', 'r') as f:
    data = json.load(f)
outputData = []
for signal in data:
    outputData.append(getCategories(signal))

with open('./tagged-signals.json', 'w') as f:
    json.dump(outputData, f)

Extracting from  113
{"signal_index":"113","categories":[{"category":"Alarm","isMatch":true,"percentageMatch":70,"keywords":["alarm","alert","notification","door bell ring"]},{"category":"Phones/Devices","isMatch":true,"percentageMatch":80,"keywords":["phone vibrating","mobile notification","vibration","phone vibration"]},{"category":"Vehicles","isMatch":true,"percentageMatch":25,"keywords":["crashing any car","video game"]},{"category":"Nature","isMatch":false,"percentageMatch":0,"keywords":[]},{"category":"Machinery","isMatch":false,"percentageMatch":0,"keywords":[]},{"category":"Music","isMatch":false,"percentageMatch":0,"keywords":[]},{"category":"Events","isMatch":true,"percentageMatch":25,"keywords":["moments before the results are announced"]},{"category":"Body","isMatch":true,"percentageMatch":50,"keywords":["heartbeat","sonography","feel the heartbeat"]},{"category":"Objects","isMatch":false,"percentageMatch":0,"keywords":[]}]}
Extracting from  114
{"signal_index":"114","categ

In [7]:
class Category(BaseModel):
    category: str
    isMatch: bool
    percentageMatch: float
    keywords: list[str]

class SignalTags(BaseModel):
    signal_index: str
    categories: list[Category]

def getEmotionalCategories(signal):
    signal_index = signal['signal_index']
    print('Extracting from ', signal_index)
    with open(query_file, 'r') as f:
        queries = json.load(f)
    descriptions = signal['emotional_descriptions']
    desc_list = '\n'.join(descriptions)
    response = client.beta.chat.completions.parse(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are an assistant that processes descriptions of haptic feedback and returns json objects"},
                {"role": "user", "content":"Signal index: "+str(signal_index)+":\n"+ queries['emotionTagPrompt']+'\n'+desc_list},
                ],
                response_format=SignalTags
        )
    print(response.choices[0].message.content)
    return json.loads(response.choices[0].message.content)

with open('../../Data/signal-descriptions/collated_descriptions.json', 'r') as f:
    data = json.load(f)
outputData = []
for signal in data:
    outputData.append(getEmotionalCategories(signal))

with open('./tagged-signals.json', 'w') as f:
    json.dump(outputData, f)

Extracting from  113
{"signal_index":"113","categories":[{"category":"Joy","isMatch":true,"percentageMatch":37.5,"keywords":["enjoyable","reassuring","calm"]},{"category":"Trust","isMatch":true,"percentageMatch":25,"keywords":["comforted","reassuring","calm"]},{"category":"Fear","isMatch":true,"percentageMatch":25,"keywords":["anxiety","anxious"]},{"category":"Surprise","isMatch":false,"percentageMatch":0,"keywords":[]},{"category":"Sadness","isMatch":false,"percentageMatch":0,"keywords":[]},{"category":"Disgust","isMatch":true,"percentageMatch":12.5,"keywords":["greasy"]},{"category":"Anger","isMatch":false,"percentageMatch":0,"keywords":[]},{"category":"Anticipation","isMatch":true,"percentageMatch":12.5,"keywords":["active","alert"]}]}
Extracting from  114
{"signal_index":"114","categories":[{"category":"Joy","isMatch":true,"percentageMatch":30,"keywords":["pleasant feeling","happy","enjoyable"]},{"category":"Trust","isMatch":false,"percentageMatch":0,"keywords":[]},{"category":"Fea

In [ ]:
import json 

with open('./tagged-signals.json', 'r') as f:
    taggedSignals = json.load(f)

categories = set()
for signal in taggedSignals:
    for category in signal['categories']:
        categories.add(category['category'])
with open('./categories.json', 'w') as f:
    json.dump(list(categories), f)

{'Calmness', 'Agitation/Anger', 'Fear', 'Confusion/Fear', 'Ambiguity', 'Soothing/Sadness', 'Trust', 'Happiness/Joy', 'Excitement/Surprise', 'Anger', 'Disgust', 'Gritty/Disgust', 'Urgency/Anticipation', 'Surprise', 'Awareness/Trust', 'Sadness', 'Anticipation', 'Joy', 'Calm', 'Tiredness/Sadness'}


In [10]:
with open('./categories.json', 'r') as f:
    categories = json.load(f)

with open('./tagged-signals.json', 'r') as f:
    taggedSignals = json.load(f)

mappings = {}
for category in categories:
    signalList = []
    for signal in taggedSignals:
        for cat in signal['categories']:
            if cat['category'] == category and cat['isMatch'] == True:
                signalList.append(signal['signal_index'])
    mappings[category] = signalList
with open('./mappings.json', 'w') as f:
    json.dump(mappings, f)